In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!cd /root/
# ! mkdir .kaggle
! cd .kaggle
! ls -l
! touch kaggle.json

In [ ]:
api_token = {"username":"kedermallick","key":"cf972c2245917c60af89c3dc85131e18"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!kaggle competitions download -c dogs-vs-cats

In [ ]:
! cd ~
! cd .kaggle/
! ls -la


In [ ]:
! cd ~
! cd /kaggle/working/ 
! ls -la
# ! unzip /root/.kaggle/dogs-vs-cats.zip -d /kaggle/working/
# ! unzip /kaggle/working/test1.zip -d /kaggle/working/
# ! unzip /kaggle/working/train.zip -d /kaggle/working/

In [ ]:
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import os

filenames=os.listdir("/kaggle/working/train")
categories=[]

for filename in filenames:
    category=filename.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})
df.tail()


In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
df['category']=df['category'].replace({
    0:'cat',
    1:'dog'
})
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

train_df,validate_df=train_test_split(df,test_size=0.20,random_state=45,shuffle=True)
train_df=train_df.reset_index(drop=True)
validate_df=validate_df.reset_index(drop=True)

df['category'].value_counts().plot.bar()

In [ ]:
train_total=train_df.shape[0]
validate_total=validate_df.shape[0]
batch_size=64

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    rescale=1./255, 
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.20
)
# train_datagen=ImageDataGenerator(
#     rescale=1./255,
# #     validation_split=0.20
# )
train_generator=train_datagen.flow_from_dataframe(
    train_df,
    "/kaggle/working/train/", 
    x_col='filename',
    y_col='category',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
# validation data prep
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/kaggle/working/train/", 
    x_col='filename',
    y_col='category',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in validation_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

In [ ]:
# define call backs 
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
earlystop=EarlyStopping(patience=20)
# reduce the learn rate if ccuracy is not increasedd in two steps
lear_rate_reduction=ReduceLROnPlateau(
    monitor='val_acc',
    patience=2,
    verbose=1,
    factor=0.5,
    min_lr=0.00001
)
callbacks = [earlystop, lear_rate_reduction]

In [ ]:
# vgg-16 network

model=Sequential()

model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1000,activation="relu"))
## as out puts can be of two types
model.add(Dense(units=2,activation="softmax"))
model.summary()

In [ ]:
# from tensorflow.keras.optimizers import Adam
# opt = Adam(learning_rate=0.001)
# model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
hist = model.fit_generator(
    steps_per_epoch=train_total//batch_size,
    generator=train_generator, 
    validation_data= validation_generator, 
    validation_steps=validate_total//batch_size,
    epochs=20,
    callbacks=callbacks
    )


In [ ]:
model.save_weights("catsdogsmodel.h5")

In [ ]:
epochs =20
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(hist.history['loss'], color='b', label="Training loss")
ax1.plot(hist.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(hist.history['acc'], color='b', label="Training accuracy")
ax2.plot(hist.history['val_acc'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
img = keras.preprocessing.image.load_img(
    "/kaggle/working/test1/10048.jpg", target_size=(224,224)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0]
# print(
#     "This image is %.2f percent cat and %.2f percent dog."
#     % (100 * (1 - score), 100 * score)
# )
print(int(score[0]))
# print(predictions)

In [ ]:
test_filenames = os.listdir("/kaggle/working/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "/kaggle/working/test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(224,224),
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

In [ ]:
# converting data into computer vision

label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

In [ ]:
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

In [ ]:
test_df['category'].value_counts().plot.bar()

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)
submission_df.to_csv('submission.csv', index=False)